# main

In [1]:
import json
import requests
import popelines
import os
from main import fix_values, populating_vals, flatten, flatten_dupe_vals
from datetime import datetime

In [2]:
def fix_values(value, key, reset_key):
    if key == reset_key:
        new_list = []
        for x in value:
            value[x][f'{reset_key}_id'] = x
            new_list.append(value[x])
        return new_list
    else:
        return value

In [3]:
# read endpoint, returns a json file of the HTTP request
def read_endpoint(endpoint, headers_set, params_set=None):
    try:
        response = requests.get(endpoint, headers=headers_set, params=params_set)
        response_text = json.loads(response.text)
        response.raise_for_status()

    except requests.exceptions.HTTPError as err:
        print(err)

    return response_text

In [4]:
# # generate all experiments given a project id
# def generate_experiments(project_id, experiment_endpoint, experiment_headers, experiment_params):
#     # get all experiments from one project
#     j_exp = read_endpoint(endpoint=experiment_endpoint, headers_set=experiment_headers, params_set=experiment_params)

#     # loop for all experiments in this project
# #     experiment_id_list = []
# #     upload_exp_list = []
# #     for exp in j_exp:
# #         exp['project_id'] = project_id
# #         upload_exp_list.append(flatten(exp, {}, ''))
# #         experiment_id_list.append(exp['id'])
        
#     return upload_exp_list, experiment_id_list

In [5]:
# generate all projects within account
def generate_projects(project_endpoint, project_headers):
    # get all projects
    j_proj = read_endpoint(endpoint=project_endpoint, headers_set=project_headers)

    # store a list of project metadata
    return j_proj

# load all projects, get project_id's

In [6]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')
pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)

# Optimizely parameters
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
    'Content-Type': 'application/json',
}

In [7]:
project_endpoint = 'https://api.optimizely.com/v2/projects'

In [ ]:
all_projects = generate_projects(project_endpoint, headers)

In [ ]:
# pope.write_to_json(file_name='../uploads/projects.json', jayson=all_projects, mode='w')

In [ ]:
# pope.write_to_bq(table_name='projects', file_name='../uploads/projects.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
# print("Successfully uploaded all projects.")

In [ ]:
project_id_list = []
for project in all_projects:
    project_id_list.append((project['id']))

In [ ]:
# project_id_list


In [ ]:
# project_id_name = []
# for project in all_projects:
#     project_id_name.append((project['id'], project['name']))

In [ ]:
# project_id_name

# load experiments

In [8]:
experiment_endpoint = 'https://api.optimizely.com/v2/experiments'


In [9]:
# experiment_id_list = []
# all_exp_list = []

# for project_id in project_id_list:
    # params include project_id (required) and experiments pulling per each request (default only 25)
params = (
    ('project_id', 10427612860),
    ('per_page', 100),
) 

#     exp_list = read_endpoint(endpoint=experiment_endpoint, headers_set=headers, params_set=params)
#     all_exp_list.extend(exp_list)

In [ ]:
exp_list = read_endpoint(endpoint=experiment_endpoint, headers_set=headers, params_set=params)



In [ ]:
# exp_list

In [ ]:
experiment_id_list = []

In [ ]:
exp_id_list = []
for exp in exp_list:
    exp_id_list.append(exp['id'])

In [ ]:
experiment_id_list.extend(exp_id_list)

In [ ]:
new = []

In [ ]:
for e in exp_list:
    if e['id'] == 12746930280:
        new = e

In [ ]:
exp=new

In [ ]:
exp

# single layer table

In [ ]:
all_singles = []
nested_key_list = []
for k,v in exp.items():
    if isinstance(v, list):
        nested_key_list.append(k)

single_layer_experiment = {}    
for k,v in exp.items():
    if k not in nested_key_list:
        k = k.replace('-', '_')
        single_layer_experiment[k] = exp[k]
single_layer_experiment['upload_ts'] = str(datetime.now())


all_singles.append(flatten(single_layer_experiment, {}, ''))


In [ ]:
all_singles

# unnested metrics table

In [ ]:
metrics_table = []
for exp in exp_list:
    print(f"Processing experiment {exp['id']}")

    # single layer fields:
    nested_key_list = []
    for k,v in exp.items():
        if isinstance(v, list) or isinstance(v, dict):
            nested_key_list.append(k)

    single_layer_experiment = {}    
    for k,v in exp.items():
        if k not in nested_key_list:
            k = k.replace('-', '_')
            single_layer_experiment[k] = exp[k]
    single_layer_experiment['upload_ts'] = str(datetime.now())

    all_singles.append(flatten(single_layer_experiment, {}, ''))

    # nested part into separate tables:
    # metrics table:

    flattened_metric = []
    for element in exp['metrics']:
        flattened_metric.append(element)

    updated_metric = populating_vals(outer_dict=exp, inner_flattened_list=flattened_metric, destination_key='metrics')
    new_flattened_metric = flatten_dupe_vals(vals=updated_metric, key='metrics')

    metric_list = []
    for metric in new_flattened_metric:
        metric_dict = {}
        metric_dict['metrics_aggregator'] = metric['metrics_aggregator']
        if 'metrics_event_id' in metric.keys():
            metric_dict['metrics_event_id'] = metric['metrics_event_id']
        metric_dict['metrics_scope'] =  metric['metrics_scope']
        metric_dict['metrics_winning_direction'] = metric['metrics_winning_direction']
        metric_dict['experiment_id'] = exp['id']
        metric_dict['upload_ts'] = str(datetime.now())
        metric_list.append(metric_dict)
        
    metrics_table.extend(metric_list)
    

In [ ]:
# nested part into separate tables:
# metrics table:

flattened_metric = []
for element in exp['metrics']:
    flattened_metric.append(element)

updated_metric = populating_vals(outer_dict=exp, inner_flattened_list=flattened_metric, destination_key='metrics')
new_flattened_metric = flatten_dupe_vals(vals=updated_metric, key='metrics')

metric_list = []
for metric in new_flattened_metric:
    metric_dict = {}
    metric_dict['metrics_aggregator'] = metric['metrics_aggregator']
    if 'metrics_event_id' in metric.keys():
        metric_dict['metrics_event_id'] = metric['metrics_event_id']
    metric_dict['metrics_scope'] =  metric['metrics_scope']
    metric_dict['metrics_winning_direction'] = metric['metrics_winning_direction']
    metric_dict['experiment_id'] = exp['id']
    metric_dict['upload_ts'] = str(datetime.now())
    metric_list.append(metric_dict)

In [ ]:
metric_list

# unnested variations table

In [ ]:
variations_table = []
for exp in exp_list:
    print(f"Processing experiment {exp['id']}")

    variations = {}
    # for exp in exp_list:

    print(f"Processing experiment {exp['id']}")

    variations['experiment_id'] = exp['id']
    variations['variations'] = exp['variations']

    flattened_variations = []

    for var in exp['variations']:
        flattened_actions = []
        if len(var['actions']) > 0:
            for action in var['actions']:
                flattened_changes = []
                for element in action['changes']:
                    flattened_changes.append(element)
                # Replace old 'changes' with new 'flattened_changes'
                updated_changes = populating_vals(outer_dict=action, inner_flattened_list=flattened_changes, destination_key='changes')
                new_flattened_changes = flatten_dupe_vals(vals=updated_changes, key='changes')

                update_actions = populating_vals(outer_dict=var, inner_flattened_list=new_flattened_changes, destination_key='actions')
                flat = flatten_dupe_vals(vals=update_actions, key='actions')
                flattened_actions.extend(flat)

        else:
            other_flat = {}
            for k,v in var.items():
                if k != 'actions':
                    other_flat['actions'] = []
                    other_flat[k] = v
            flat = [other_flat]
            flattened_actions.extend(flat)

        update_variations = populating_vals(outer_dict=variations, inner_flattened_list=flattened_actions, destination_key='variations')
        flattened_variations.extend(flatten_dupe_vals(vals=update_variations, key='variations'))

    variations_table.extend(flattened_variations)
    
    

In [ ]:
variations = {}
# for exp in exp_list:

print(f"Processing experiment {exp['id']}")

variations['experiment_id'] = exp['id']
variations['variations'] = exp['variations']



In [ ]:
exp['variations'][0]


In [ ]:
flattened_variations = []

for var in exp['variations']:
    flattened_actions = []  
    
    if len(var['actions']) > 0:
        for action in var['actions']:

            flattened_changes = []

            if action['changes'] != []:
                for element in action['changes']:
                    flattened_changes.append(element)
                # Replace old 'changes' with new 'flattened_changes'
                updated_changes = populating_vals(outer_dict=action, inner_flattened_list=flattened_changes, destination_key='changes')
                new_flattened_changes = flatten_dupe_vals(vals=updated_changes, key='changes')

                update_actions = populating_vals(outer_dict=var, inner_flattened_list=new_flattened_changes, destination_key='actions')
                flat = flatten_dupe_vals(vals=update_actions, key='actions')
                flattened_actions.extend(flat)

            else:
                new_actions = flatten_dupe_vals(vals=var, key='actions')
                flattened_actions.extend(new_actions)
            
            print('completed round 1')
            
    else:
        other_flat = {}
        for k,v in var.items():
            if k != 'actions':
                other_flat['actions'] = []
                other_flat[k] = v
        flat = [other_flat]
        flattened_actions.extend(flat)

        print('completed round 2')

    update_variations = populating_vals(outer_dict=variations, inner_flattened_list=flattened_actions, destination_key='variations')
    flattened_variations.extend(flatten_dupe_vals(vals=update_variations, key='variations'))

    
    

In [ ]:
flattened_variations

### Testing flatten_dupe_vals

In [ ]:
# new_flattened_test = flatten_dupe_vals(vals=test, key='actions')
vals = test
key = 'actions'

In [ ]:
duped_results = []
for element in vals[key]:
    ts_dict = populating_vals(outer_dict=vals, inner_flattened_list=element, destination_key=key)
    duped_results.append(ts_dict)



In [ ]:
duped_results

In [ ]:
flattened_results = []
for element in duped_results:
    flattened_results.append(flatten(element, {}, ''))

In [ ]:
flatten(duped_results[0], {}, '')

### It is the flatten function that f*cked things up

In [ ]:
duped_results[0]

In [ ]:
# corrected flatten
def flatten(jayson, acc, prefix):
    if isinstance(jayson, dict):
        for k,v in jayson.items():
            if prefix:
                prefix_k = prefix + "_" + k
            else: 
                prefix_k = k
            prefix_k = prefix_k.replace('-', '_')
            
            if isinstance(v, dict):
                flatten(v, acc, prefix_k)
            elif isinstance(v, list):
                if v != []:
                    for j in v:
                        flatten(j, acc, prefix_k)
                else:
                    acc[prefix_k] = v
            else:
                acc[prefix_k] = v
        return acc 
    else:
        return acc

In [ ]:
flatten(duped_results[0], {}, '')

### Thus fixing the missing changes issue!

In [ ]:
flattened_variations = []

for var in expanded_exp['variations']:
    flattened_actions = []
    for action in var['actions']:
        print(action)
#         flattened_changes = []
#         for element in action['changes']:
#             flattened_changes.append(element)

        # Replace old 'timeseries' with new 'flattened_timeseries'
#         updated_changes = populating_vals(outer_dict=action, inner_flattened_list=flattened_changes, destination_key='changes')
#         new_flattened_changes = flatten_dupe_vals(vals=updated_changes, key='changes')

#     update_actions = populating_vals(outer_dict=var, inner_flattened_list=new_flattened_changes, destination_key='actions')
#     flattened_actions.extend(flatten_dupe_vals(vals=update_actions, key='actions'))  

# update_variations = populating_vals(outer_dict=expanded_exp, inner_flattened_list=flattened_actions, destination_key='variations')
# flattened_variations.extend(flatten_dupe_vals(vals=update_variations, key='variations'))

# expanded_variations = flattened_variations

In [ ]:
flattened_changes

# results timeseries

In [ ]:
# experiment_id_list

In [70]:
# example of no results
# result_endpoint = 'https://api.optimizely.com/v2/experiments/11479333433/timeseries'
# example of normal time series
result_endpoint = 'https://api.optimizely.com/v2/experiments/12813120687/timeseries'
# example of 
# result_endpoint = f'https://api.optimizely.com/v2/experiments/{experiment_id}/timeseries'



In [72]:
# for experiment_id in experiment_id_list:
# result_endpoint = f'https://api.optimizely.com/v2/experiments/{experiment_id}/timeseries'
response_ts = requests.get(result_endpoint, headers=headers)
print(f"got experiment 12813120687")
    
    

got experiment 12813120687


In [16]:
# if '' then the experiment has not started yet
if response_ts.text == '' or 'bad' in response_ts.text:
    j_ts = {'experiment_id': experiment_id}
    new_j_ts = j_ts
    new_j_ts['upload_ts'] = str(datetime.now())
    
else:
    j_ts = json.loads(response_ts.text)

    new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')

    flattened_j_ts = []

    flattened_metrics = []
    for metric in new_j_ts['metrics']:
        if 'results' in metric.keys():
            
            for ts in metric['results']:
                flattened_timeseries = []
                for element in ts['timeseries']:
                    element['upload_ts'] = str(datetime.now())
                    flattened_timeseries.append(flatten(element, {}, ''))

                # Replace old 'timeseries' with new 'flattened_timeseries'
                updated_results = populating_vals(outer_dict=ts, inner_flattened_list=flattened_timeseries, destination_key='timeseries')
                flattened_results = flatten_dupe_vals(vals=updated_results, key='timeseries')

            # Replace old 'metrics' with new 'flattened_results'
            update_metrics = populating_vals(outer_dict=metric, inner_flattened_list=flattened_results, destination_key='results')
            flattened_metrics.extend(flatten_dupe_vals(vals=update_metrics, key='results'))

        else:
            flattened_metrics = [flatten(new_j_ts, {}, '')]

    update_new_j_ts = populating_vals(outer_dict=new_j_ts, inner_flattened_list=flattened_metrics, destination_key='metrics')
    flattened_j_ts.extend(flatten_dupe_vals(vals=update_new_j_ts, key='metrics'))
    
    

B


In [73]:
j_ts = json.loads(response_ts.text)

new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')





In [74]:
new_j_ts['metrics']

[{'aggregator': 'unique',
  'event_id': 12097935537,
  'name': '[Demo] Infusionsoft Pro Demo Submit',
  'results': [{'is_baseline': True,
    'level': 'variation',
    'name': 'Drift',
    'results_id': '12798060466',
    'timeseries': [{'rate': 0.047619047619047616,
      'samples': 21,
      'time': '2019-01-08T23:29:23.000000Z',
      'value': 1,
      'variance': 0.04535147392290249},
     {'rate': 0.07692307692307693,
      'samples': 18,
      'time': '2019-01-09T01:39:32.370000Z',
      'value': 2,
      'variance': 0.07100591715976332},
     {'rate': 0.0784313725490196,
      'samples': 12,
      'time': '2019-01-09T03:49:41.740000Z',
      'value': 1,
      'variance': 0.0722798923490965},
     {'rate': 0.06779661016949153,
      'samples': 8,
      'time': '2019-01-09T05:59:51.110000Z',
      'value': 0,
      'variance': 0.06320022981901752},
     {'rate': 0.06060606060606061,
      'samples': 7,
      'time': '2019-01-09T08:10:00.480000Z',
      'value': 0,
      'variance'

In [75]:
flattened_j_ts = []

flattened_metrics = []
for metric in new_j_ts['metrics']:
    if 'results' in metric.keys():
        flattened_results = []
        for ts in metric['results']:
            flattened_timeseries = []
            for element in ts['timeseries']:
                element['upload_ts'] = str(datetime.now())
                flattened_timeseries.append(flatten(element, {}, ''))

            # Replace old 'timeseries' with new 'flattened_timeseries'
            updated_results = populating_vals(outer_dict=ts, inner_flattened_list=flattened_timeseries, destination_key='timeseries')
            flattened_results.extend(flatten_dupe_vals(vals=updated_results, key='timeseries'))

        # Replace old 'metrics' with new 'flattened_results'
        update_metrics = populating_vals(outer_dict=metric, inner_flattened_list=flattened_results, destination_key='results')
        flattened_metrics.extend(flatten_dupe_vals(vals=update_metrics, key='results'))

    else:
        flattened_metrics = [flatten(new_j_ts, {}, '')]

update_new_j_ts = populating_vals(outer_dict=new_j_ts, inner_flattened_list=flattened_metrics, destination_key='metrics')
flattened_j_ts.extend(flatten_dupe_vals(vals=update_new_j_ts, key='metrics'))


In [57]:

# for ts in new_j_ts['metrics'][0]['results']:
#     flattened_timeseries = []
#     for element in ts['timeseries']:
#         element['upload_ts'] = str(datetime.now())
#         flattened_timeseries.append(flatten(element, {}, ''))
#         break

#     # Replace old 'timeseries' with new 'flattened_timeseries'
#     updated_results = populating_vals(outer_dict=ts, inner_flattened_list=flattened_timeseries, destination_key='timeseries')
#     flattened_results.extend(flatten_dupe_vals(vals=updated_results, key='timeseries'))
#     break

In [76]:
len(flattened_j_ts)

600

In [82]:
output = []
for e in flattened_j_ts:
    if e['metrics_name'] == 'Modal open button' and e['metrics_results_timeseries_time'] == '2019-01-08T23:29:23.000000Z':
        output.append(e)
        
        

In [83]:
output


[{'confidence_threshold': 0.9,
  'end_time': '2019-01-18T00:25:00.000000Z',
  'experiment_id': 12813120687,
  'metrics_aggregator': 'unique',
  'metrics_event_id': 12837180433,
  'metrics_name': 'Modal open button',
  'metrics_results_is_baseline': True,
  'metrics_results_level': 'variation',
  'metrics_results_name': 'Drift',
  'metrics_results_results_id': '12798060466',
  'metrics_results_timeseries_rate': 0.047619047619047616,
  'metrics_results_timeseries_samples': 21,
  'metrics_results_timeseries_time': '2019-01-08T23:29:23.000000Z',
  'metrics_results_timeseries_upload_ts': '2019-02-21 15:55:33.920374',
  'metrics_results_timeseries_value': 1,
  'metrics_results_timeseries_variance': 0.04535147392290249,
  'metrics_results_variation_id': '12798060466',
  'metrics_scope': 'visitor',
  'metrics_winning_direction': 'increasing',
  'start_time': '2019-01-08T23:29:23.000000Z',
  'stats_config_confidence_level': 0.9,
  'stats_config_difference_type': 'relative',
  'stats_config_epoc

In [29]:
length = 0
for e in new_j_ts['metrics'][0]['results']:
    print(f"{e['name']} is of {len(e['timeseries'])} long")

Variation #2 is of 100 long
Variation #1 is of 100 long
Original is of 100 long


In [26]:
length

300

In [31]:
for e in flattened_j_ts:
    print(e['metrics_results_name'])

Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original
Original


# testing

In [ ]:
from generate_original import read_endpoint, generate_projects, generate_experiments

In [ ]:
if not os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS'):
    os.environ['GOOGLE_ACCOUNT_CREDENTIALS'] = '/home/engineering/keyfile.json'
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')

############################################### Instantiating Popelines #######################################
pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)

# Optimizely parameters
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}

# endpoints
project_endpoint = 'https://api.optimizely.com/v2/projects'
experiment_endpoint = 'https://api.optimizely.com/v2/experiments'


############################################### generate and upload all projects ##############################
all_projects = generate_projects(project_endpoint, headers)
for project in all_projects:
    project['upload_ts'] = str(datetime.now())

# upload projects 
pope.write_to_json(file_name='../uploads/projects.json', jayson=all_projects, mode='w')
pope.write_to_bq(table_name='projects', file_name='../uploads/projects.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)


############################################### generate and upload all experiments ##############################
# get a list of project_id from all_projects
project_id_list = []
for project in all_projects:
    project_id_list.append(project['id'])

# to accumulate all experiment_id, for 
experiment_id_list = []
origin_single_table = []
origin_metrics_table = []
origin_variations_table = []

# loop over all project_id_list to get experiments within each project
for project_id in project_id_list:
    # params include project_id (required) and experiments pulling per each request (default only 25)
    params = (
        ('project_id', project_id),
        ('per_page', 100),
    ) 

    exp_list = read_endpoint(endpoint=experiment_endpoint, headers_set=headers, params_set=params)
    exp_id_list = []
    for exp in exp_list:
        exp_id_list.append(exp['id'])
    experiment_id_list.extend(exp_id_list)

    all_singles, metrics_table, variations_table = generate_experiments(exp_list)
    origin_single_table.extend(all_singles)
    origin_metrics_table.extend(metrics_table)
    origin_variations_table.extend(variations_table)

pope.write_to_json(file_name='../uploads/origin_experiments_single_fields.json', jayson=origin_single_table, mode='w')  
pope.write_to_json(file_name='../uploads/origin_experiments_metrics_table.json', jayson=origin_metrics_table, mode='w')
pope.write_to_json(file_name='../uploads/origin_experiments_variations_table.json', jayson=origin_variations_table, mode='w')





In [ ]:
for e in origin_single_table:
    if e['id'] == 12900000180:
        print(e)

In [ ]:
for m in origin_metrics_table:
    if m['experiment_id'] == 12900000180:
        print(m)

In [ ]:
l = []
for v in origin_variations_table:
    if v['experiment_id'] == 12900000180:
        l.append(v['variations_name'])

In [ ]:
l

In [ ]:
exp_list

In [ ]:
for e in exp_list:
    print(e['id'])

In [ ]:
exp

In [ ]:
variations = {}
variations['experiment_id'] = exp['id']
variations['variations'] = exp['variations']

flattened_variations = []

for var in exp['variations']:
    flattened_actions = []
    if len(var['actions']) > 0:
        for action in var['actions']:
            flattened_changes = []
            for element in action['changes']:
                flattened_changes.append(element)
            # Replace old 'changes' with new 'flattened_changes'
            updated_changes = populating_vals(outer_dict=action, inner_flattened_list=flattened_changes, destination_key='changes')
            new_flattened_changes = flatten_dupe_vals(vals=updated_changes, key='changes')

        update_actions = populating_vals(outer_dict=var, inner_flattened_list=new_flattened_changes, destination_key='actions')
        flat = flatten_dupe_vals(vals=update_actions, key='actions')
        
        flattened_actions.extend(flat)

    else:
        other_flat = {}
        for k,v in var.items():
            if k != 'actions':
                other_flat['actions'] = []
                other_flat[k] = v
        flat = [other_flat]
    
        flattened_actions.extend(flat)

    update_variations = populating_vals(outer_dict=variations, inner_flattened_list=flattened_actions, destination_key='variations')
    flattened_variations.extend(flatten_dupe_vals(vals=update_variations, key='variations'))
    

In [ ]:
len(flattened_variations)

In [ ]:
for v in flattened_variations:
    print(v['variations_variation_id'])

In [ ]:
for v in flattened_variations:
    print(v['variations_name'])

In [ ]:
for v in flattened_variations:
    if 'variations_actions_changes_id' in v.keys():
        print(v['variations_actions_changes_id'])